# Analisis del equilibrio

## Importación de librerías y lectura de datos

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
codigos=["110-160","160-210","110-160","210-260","260-310","310-360"]
DFS_vx=[]
DFS_vy=[]
DFS_vz=[]

DATOS=[]

for codigo in codigos:
    DFS_vx.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/velocidad_x.txt", delim_whitespace=True, header=None))
    DFS_vy.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/velocidad_y.txt", delim_whitespace=True, header=None))
    DFS_vz.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/velocidad_z.txt", delim_whitespace=True, header=None))
    DATOS.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/gname2.txt",skiprows=1, delim_whitespace=True, header=None, names=['t','Ec','Ep','E_tot','W','T','P_corr','F2']))

/var/folders/15/rn_wb7ms2yq2yn7zj6rqkq880000gn/T/ipykernel_9162/3023447748.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  DFS_vx.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/velocidad_x.txt", delim_whitespace=True, header=None))
/var/folders/15/rn_wb7ms2yq2yn7zj6rqkq880000gn/T/ipykernel_9162/3023447748.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  DFS_vy.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/velocidad_y.txt", delim_whitespace=True, header=None))
/var/folders/15/rn_wb7ms2yq2yn7zj6rqkq880000gn/T/ipykernel_9162/3023447748.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  DFS_vz.append(pd.read_csv("../resultados/dinamica/Simulacion_"+

In [21]:
tiempos=[df.t.to_numpy() for df in DATOS]

tiempos[0]

array([1.000e-02, 2.000e-02, 3.000e-02, ..., 4.998e+01, 4.999e+01,
       5.000e+01], shape=(5000,))

### Formato de datos importados

- En las columnas tenemos las particulas de la caja etiquetadas del 0 al 499
- En las filas tenemos instantes sucesivos de tiempo grabados de nuestra simulacion.

In [38]:
DATOS[4].iloc[:,1:].agg(["mean","std"])

,Ec,Ep,E_tot,W,T,P_corr,F2
mean,1024.195928,-1689.225999,-665.030063,-1238.774635,1.365595,0.236364,571.458929
std,17.646443,17.651434,0.026460,341.948152,0.023529,0.108351,82.228914


In [56]:
F2s_mean=[]
F2s_std=[]

Ecs_mean=[]
Ecs_std=[]

Ps_mean=[]
Ps_std=[]


for df in DATOS:
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","F2"])
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","F2"])

    Ecs_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","Ec"])
    Ecs_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","Ec"])

    Ps_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","P_corr"])
    Ps_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","P_corr"])


F2s_mean=np.array(F2s_mean)
F2s_std=np.array(F2s_mean)

Ecs_mean=np.array(Ecs_mean)
Ecs_std=np.array(Ecs_std)

Ps_mean=np.array(Ps_mean)
Ps_std=np.array(Ps_std)


"""
print("Ec=",)

print(F2s_mean)
print(F2s_std)
print(Ecs_mean)
print(Ecs_std)
print(Ps_mean)
print(Ps_std)

"""

# Media global = media de las medias
F2_mean = np.mean(F2s_mean)
Ec_mean = np.mean(Ecs_mean)
P_mean  = np.mean(Ps_mean)

# Error estándar de la media global --> ¿Deberia tener en cuenta las std de cada bloque simulado?
F2_err = np.std(F2s_mean, ddof=1) / np.sqrt(len(F2s_mean))
Ec_err = np.std(Ecs_mean, ddof=1) / np.sqrt(len(Ecs_mean))
P_err  = np.std(Ps_mean, ddof=1) / np.sqrt(len(Ps_mean))

print(f"<F²> = {F2_mean:.6f} ± {F2_err:.6f}")
print(f"<Ec> = {Ec_mean:.6f} ± {Ec_err:.6f}")
print(f"<P>  = {P_mean:.6f} ± {P_err:.6f}")

<F²> = 326.931285 ± 73.672704
<Ec> = 1023.071370 ± 0.898193
<P>  = 0.233027 ± 0.002185
